### create a python code that integrates the multimodel from the amazon bedrock and perform the specific operations to the given image

In [ ]:
import boto3
import json
import base64
import io
from PIL import Image
import uuid


bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

model_id = "amazon.nova-canvas-v1:0"

prompt = "Create a image of beautiful natural view"

body = json.dumps({
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 1024,
        "width": 1024,
        "cfgScale": 8.0,
        "seed": 0
    }
})


accept = "application/json"
content_type = "application/json"

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    accept=accept, 
    contentType=content_type
)

result = json.loads(response["body"].read())
base64_image = result.get("images")[0]  
image_bytes = base64.b64decode(base64_image)  

image = Image.open(io.BytesIO(image_bytes))
image.show()
id=uuid.uuid4()
object_path=f"Generated_images/{id}.png"
save_path = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/{object_path}"  # Change this path as needed
image.save(save_path)

print(f"Image successfully saved at: {save_path}")

final_path=save_path



def process_image_with_prompt(image_path, prompt):
    """Processes an existing image using Amazon Nova Lite."""
    model_id = "amazon.titan-image-generator-v2:0"

    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    payload = {
        "taskType": "IMAGE_TRANSFORMATION",
        "textToImageParams": {"text": prompt},
        "imageToImageParams": {"image": base64_image},
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": 0
        }
    }

    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(payload),
        accept="application/json",
        contentType="application/json"
    )

    result = json.loads(response["body"].read())
    modified_image_base64 = result.get("images")[0]

    # Decode and save modified image
    image_bytes = base64.b64decode(modified_image_base64)
    image = Image.open(io.BytesIO(image_bytes))
    
    modified_image_id = str(uuid.uuid4())
    output_path = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/Generated_images/{modified_image_id}.png"
    image.save(output_path)
    image.show()

    return output_path

modify_prompt = "Make the sky more colorful and add some birds"
modified_image_path = process_image_with_prompt(final_path, modify_prompt)









# ## Save the image to the S3


# s3 = boto3.client("s3")
# id = uuid.uuid4()

# bucket_name="myawsimgbucket-meet"


# response=s3.upload_file(f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/{object_path}",
#                         bucket_name,
#                         object_path,
#                         ExtraArgs={'ContentType': 'image/jpeg'}
#                         )

# response = s3.put_object(
#             Bucket=bucket_name,
#             Key=object_path,
#             Body=image_bytes,
#             ContentType='image/jpeg'
#         )

# object_url = f"https://{bucket_name}.s3.us-east-1.amazonaws.com/{object_path}"
# print(object_url)
# print("Link has been Generated")


In [ ]:
import boto3
bedrock_client = boto3.client("bedrock", region_name="us-east-1")
response = bedrock_client.list_foundation_models()
print(response)

In [ ]:
import boto3
import json
import base64
import io
from PIL import Image
import uuid

# Initialize Amazon Bedrock client
bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Amazon Titan Image Generator v2 model
model_id = "amazon.titan-image-generator-v2:0"

# Step 1: Generate Initial Image
initial_prompt = "team celebrating the vistory of the world cup"
payload = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {"text": initial_prompt},
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 1024,
        "width": 1024,
        "cfgScale": 8.0,
        "seed": 0
    }
}

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=json.dumps(payload),
    accept="application/json",
    contentType="application/json"
)

result = json.loads(response["body"].read())
base64_image = result.get("images")[0]
image_bytes = base64.b64decode(base64_image)

# Save and display the initial image
initial_image = Image.open(io.BytesIO(image_bytes))
initial_image.show()

initial_image_path = f"generated_image_{uuid.uuid4()}.png"
initial_image.save(initial_image_path)
print(f"Initial image saved at: {initial_image_path}")

# Step 2: Modify the Image (by regenerating with a better prompt)
modified_prompt = "Change all five players to male in the previously generated photo with a trophy winning celebration."
payload["textToImageParams"]["text"] = modified_prompt  # Update prompt

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=json.dumps(payload),
    accept="application/json",
    contentType="application/json"
)

result = json.loads(response["body"].read())
base64_modified_image = result.get("images")[0]
modified_image_bytes = base64.b64decode(base64_modified_image)

# Save and display the modified image
modified_image = Image.open(io.BytesIO(modified_image_bytes))
modified_image.show()

modified_image_path = f"modified_image_{uuid.uuid4()}.png"
modified_image.save(modified_image_path)
print(f"Modified image saved at: {modified_image_path}")


In [ ]:
# import boto3
# import json
# import base64
# import io
# from PIL import Image
# import uuid
# import os

# # Initialize Amazon Bedrock Client
# bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")  

# # S3 Client
# s3 = boto3.client("s3")

# # S3 Bucket Name (Ensure this exists in your AWS)
# bucket_name = "myawsimgbucket-meet"

# # **Generate Image from Text**
# def generate_image(prompt):
#     """Generates an image using Amazon Nova Canvas."""
#     model_id = "amazon.nova-canvas-v1:0"

#     body = json.dumps({
#         "taskType": "TEXT_IMAGE",
#         "textToImageParams": {
#             "text": prompt
#         },
#         "imageGenerationConfig": {
#             "numberOfImages": 1,
#             "height": 1024,
#             "width": 1024,
#             "cfgScale": 8.0,
#             "seed": 0
#         }
#     })

#     response = bedrock_client.invoke_model(
#         modelId=model_id,
#         body=body,
#         accept="application/json",
#         contentType="application/json"
#     )

#     # Extract the generated image
#     result = json.loads(response["body"].read())
#     base64_image = result.get("images")[0]
#     image_bytes = base64.b64decode(base64_image)

#     # Open and save image locally
#     image = Image.open(io.BytesIO(image_bytes))
#     image_id = str(uuid.uuid4())
#     local_folder = "C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/Generated_images"
    
#     # Ensure directory exists
#     os.makedirs(local_folder, exist_ok=True)

#     local_path = f"{local_folder}/{image_id}.png"
#     image.save(local_path)

#     print(f"Image successfully saved at: {local_path}")

#     return local_path, image_bytes, image_id

# # **Upload Image to S3**
# def upload_to_s3(local_path, image_bytes, image_id):
#     """Uploads an image to S3 and returns the S3 URL."""
#     s3_key = f"Generated_images/{image_id}.png"

#     s3.put_object(
#         Bucket=bucket_name,
#         Key=s3_key,
#         Body=image_bytes,
#         ContentType='image/png'
#     )

#     s3_url = f"https://{bucket_name}.s3.us-east-1.amazonaws.com/{s3_key}"
#     print(f"Image uploaded to S3: {s3_url}")
#     return s3_url

# # **Modify Image with Multi-Modal Model**
# def process_image_with_prompt(image_path, prompt):
#     """Processes an existing image using Amazon Nova Lite."""
#     model_id = "amazon.nova-lite-v1"

#     with open(image_path, "rb") as image_file:
#         base64_image = base64.b64encode(image_file.read()).decode("utf-8")

#     payload = {
#         "taskType": "IMAGE_TRANSFORMATION",
#         "textToImageParams": {"text": prompt},
#         "imageToImageParams": {"image": base64_image},
#         "imageGenerationConfig": {
#             "numberOfImages": 1,
#             "height": 1024,
#             "width": 1024,
#             "cfgScale": 8.0,
#             "seed": 0
#         }
#     }

#     response = bedrock_client.invoke_model(
#         modelId=model_id,
#         body=json.dumps(payload),
#         accept="application/json",
#         contentType="application/json"
#     )

#     result = json.loads(response["body"].read())
#     modified_image_base64 = result.get("images")[0]

#     # Decode and save modified image
#     image_bytes = base64.b64decode(modified_image_base64)
#     image = Image.open(io.BytesIO(image_bytes))
    
#     modified_image_id = str(uuid.uuid4())
#     output_path = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/Generated_images/{modified_image_id}.png"
#     image.save(output_path)
#     image.show()

#     return output_path

# # **Main Execution**
# if __name__ == "__main__":
#     prompt_text = "Create an image of sea where it has trees on the beach, mountains, and birds flying in the sky"
    
#     # **Step 1: Generate Image**
#     generated_image_path, generated_image_bytes, generated_image_id = generate_image(prompt_text)

#     # **Step 2: Upload to S3**
#     s3_link = upload_to_s3(generated_image_path, generated_image_bytes, generated_image_id)

#     # **Step 3: Modify Image with Multi-Modal Model**
#     modify_prompt = "Make the sky more colorful and add some birds"
#     modified_image_path = process_image_with_prompt(generated_image_path, modify_prompt)

#     # **Step 4: Upload Modified Image to S3**
#     modified_s3_link = upload_to_s3(modified_image_path, modified_image_bytes, modified_image_id)

#     print(f"✅ Original Image URL: {s3_link}")
#     print(f"✅ Modified Image URL: {modified_s3_link}")


In [ ]:
# import boto3
# import json
# import base64
# import io
# from PIL import Image

# # Initialize Amazon Bedrock Client
# bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")  # Change to your AWS region

# # Model ID for Multi-Modal Processing (Supports Image & Text)
# model_id = "amazon.nova-lite-v1"

# def process_image_with_prompt(image_path, prompt):

#     with open(image_path, "rb") as image_file:
#         base64_image = base64.b64encode(image_file.read()).decode("utf-8")

#     payload = {
#         "taskType": "IMAGE_TRANSFORMATION",  
#         "textToImageParams": {"text": prompt},
#         "imageToImageParams": {"image": base64_image},
#         "imageGenerationConfig": {
#             "numberOfImages": 1,
#             "height": 1024,
#             "width": 1024,
#             "cfgScale": 8.0,
#             "seed": 0
#         }
#     }

#     response = bedrock_client.invoke_model(
#         modelId=model_id,
#         body=json.dumps(payload),
#         accept="application/json",
#         contentType="application/json"
#     )

#     result = json.loads(response["body"].read())
#     modified_image_base64 = result.get("images")[0]

#     image_bytes = base64.b64decode(modified_image_base64)
#     image = Image.open(io.BytesIO(image_bytes))
#     id2=uuid.uuid4()
#     save_path = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/{object_path}"  # Change this path as needed

#     output_path = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/Generated_images/{id2}.png"
#     image.save(output_path)
#     image.show()

#     return output_path

# id=uuid.uuid4()

# object_path=f"Generated_images/{id}.png"

# # Example Usage
# input_image = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/{object_path}"  # Replace with your image path
# prompt_text = "Make the sky more colorful and add some birds"
# save_path = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/{object_path}"  # Change this path as needed

# output_file = process_image_with_prompt(input_image, prompt_text)

# print(f"Processed image saved at: {output_file}")
